# Inflation Curves Override CPIs & Parameterization

This notebook demonstrates how to access and use the functionalities of **Inflation Curves** within the **LSEG Financial Analytics SDK**. 

Inflation curves provide forward Consumer Price Index data used for pricing inflation indexed instruments.

**You will be able to:**
- Override CPIs for inflation curve data
- configure curve parameters (interpolation, look back month etc.)

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.market_data.inflation_curves` - To access inflation curves data

This notebook uses external libraries **pandas, matplotlib**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.market_data import inflation_curves as infc
import json
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

## 1. Override CPIs in the curve data response

We first build an inflation curve where some curve points are overriddent. It is built according to the 3-step logic definition/parameters/request where **consumer price indices (CPI) are overridden in parameters**. In the results, the CPIs for January 2025 and February 2025 are overriden

### Data Preparation

In [2]:
print("Step 1: Creating Curve Definition...")
# Select an Inflation Index
inflation_index = 'AGBRPI'

# Create curve definition object
curve_definition = infc.InflationCurveDefinitionItem(
        inflation_index = infc.InflationIndex(code=inflation_index),
        )
print(f"   ✓ Instrument: {curve_definition.inflation_index}")

print("Step 2: Configuring Curve Parameters...")
# Create CPIs for the wanted month and year to be overriden
cpi_jan_2025 = infc.ConsumerPriceIndex(index_value=315, month=infc.MonthEnum.JANUARY, year=2025)
cpi_feb_2025 = infc.ConsumerPriceIndex(index_value=315, month=infc.MonthEnum.FEBRUARY, year=2025)

curve_parameters = infc.InflationCurveParameters(
        valuation_date_time = dt.datetime.strptime("2025-01-18", "%Y-%m-%d"),
        consumer_price_indexes = [cpi_jan_2025, cpi_feb_2025]
    )
print(f"   ✓ Curve Parameters: {curve_parameters}")

print("Step 3: Create request item...")
# Create the main request object  with basic configuration
request_item = infc.InflationCurvesRequestItem(
        curve_tag = f"{inflation_index}_InflationCurve",
        curve_definition = curve_definition,
        curve_parameters = curve_parameters,
    )
print(f"   ✓ Request Item: {json.dumps(request_item.as_dict(), indent=4)}")

Step 1: Creating Curve Definition...
   ✓ Instrument: {'code': 'AGBRPI'}
Step 2: Configuring Curve Parameters...
   ✓ Curve Parameters: {'valuationDateTime': '2025-01-18T00:00:00Z', 'consumerPriceIndexes': [{'indexValue': 315, 'month': 'January', 'year': 2025}, {'indexValue': 315, 'month': 'February', 'year': 2025}]}
Step 3: Create request item...
   ✓ Request Item: {
    "curveTag": "AGBRPI_InflationCurve",
    "curveDefinition": {
        "inflationIndex": {
            "code": "AGBRPI"
        }
    },
    "curveParameters": {
        "valuationDateTime": "2025-01-18T00:00:00Z",
        "consumerPriceIndexes": [
            {
                "indexValue": 315,
                "month": "January",
                "year": 2025
            },
            {
                "indexValue": 315,
                "month": "February",
                "year": 2025
            }
        ]
    }
}


### Request Execution

In [3]:
# Execute the calculation using the calculate function
# The 'universe' parameter accepts a list of request items for batch processing
response = infc.calculate(universe=[request_item])
curve_data = response['data'][0]

### Results Display

In [4]:
cpis = curve_data['curves']['consumerPriceIndexCurvePoints']
cpis_df = pd.DataFrame(cpis)
cpis_df["Month/Year"] = cpis_df['month'] + ' ' + cpis_df['year'].astype(str)
del cpis_df['month'],cpis_df['year']
cpis_df.set_index("Month/Year", inplace = True)
cpis_df.head(5) # remove .head(5) to get the entire dataframe

,indexValue,isOverridden
Month/Year,,
November 2024,390.900000,NaN
December 2024,392.100000,NaN
January 2025,315.000000,True
February 2025,315.000000,True
March 2025,324.460248,NaN


## 2. Configure more parameters

Here, starting from the previous curve we configure more parameters for the inflation curve. In the results, one can see that the curves starts from December 2024 instead of October 2024, which is the result of setting look back month at 1 instead of the default one for AGBRPI which is 3

### Data Preparation

In [5]:
curve_parameters.interpolation_mode = "Step" # Possible values: Step or Linear
curve_parameters.look_back_month = 1 # Number of months to go back in historical CPIs
curve_parameters.price_side = infc.CurvesAndSurfacesPriceSideEnum.BID # Bid, Ask or Mid

### Request Execution

In [6]:
response = infc.calculate(universe=[request_item])
curve_data = response['data'][0]

### Results Display

In [7]:
cpis = curve_data['curves']['consumerPriceIndexCurvePoints']
cpis_df = pd.DataFrame(cpis)
cpis_df["Month/Year"] = cpis_df['month'] + ' ' + cpis_df['year'].astype(str)
del cpis_df['month'],cpis_df['year']
cpis_df.set_index("Month/Year", inplace = True)
cpis_df.head(5) # remove .head(5) to get the entire dataframe

,indexValue,isOverridden
Month/Year,,
December 2024,392.100000,NaN
January 2025,315.000000,True
February 2025,315.000000,True
March 2025,323.583535,NaN
April 2025,335.096198,NaN
